In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
from datetime import datetime, date
import requests
from xgrads import open_CtlDataset
import geopandas as gpd
import matplotlib.pyplot as plt
from array import array


In [6]:
########## USER ###########
# Select modeling domain
domain = 'BEAU'

# SM location 
SMpath = '/nfs/attic/dfh/2020_NPRB/domain_'+domain+'/snowmodel2023_cfsv2/'
#SMpath = '/scratch/Nina/NPRB/domain_'+domain+'/snowmodel2023_cfsv2/'
# SM output file location
SMpathOUT = SMpath+'outputs_fut/'

# # updated roff .gdat file
# rofxgdatout = SMpath+'outputs/wo_assim/rofx_cal.gdat'
# # updated roff .ctl file
# rofxctlout = SMpath+'ctl_files/wo_assim/rofx_cal.ctl'
# # updated et .gdat file
# etxgdatout= SMpath+'outputs/wo_assim/etx_cal.gdat'
# # updated et .ctl file
# etxctlout = SMpath+'ctl_files/wo_assim/etx_cal.ctl'

#path to CSO domains
domains_resp = requests.get("https://raw.githubusercontent.com/NPRB/02_preprocess_python/main/NPRB_domains.json")
domains = domains_resp.json()

# domain parameters
nx = domains[domain]['ncols']
ny = domains[domain]['nrows']
clsz = domains[domain]['cellsize']
xll = domains[domain]['xll']
yll = domains[domain]['yll']

start_years = list(range(1989,2019))
start_years

[1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018]

In [7]:
for styr in start_years:
    print(styr)
    #start calibration date    
    st_dt = str(styr)+'-10-01'
    #end calibration date
    ed_dt = str(styr+2)+'-09-30'
    

    nt = (datetime.strptime(ed_dt,'%Y-%m-%d')-datetime.strptime(st_dt,'%Y-%m-%d')).days+1

    # file naming/organization 
    yrlabel = str(styr+81)+'-'+str(styr+82)

    # modified runoff ctl
    rofxctlout = SMpathOUT+'rofx_'+yrlabel+'.ctl'
    # modified et ctl
    etxctlout = SMpathOUT+'etx_'+yrlabel+'.ctl'
    
    # Write out .ctl file for hydroflow steps
    dt = datetime.strptime(st_dt, '%Y-%m-%d').strftime('%M:%SZ%d%b%Y')
    
    lines = ['DSET ^rofx_'+yrlabel+'.gdat',\
             'UNDEF -9999.0',\
             'XDEF '+nx+' LINEAR '+xll+' '+clsz,\
             'YDEF '+ny+' LINEAR '+yll+' '+clsz,\
             'ZDEF 1 LINEAR 1 1',\
             'TDEF '+str(nt)+' LINEAR '+dt+' 1dy',\
             'VARS 1',\
             'roff 0 0 runoff from snowpack base (m/time_step)',\
             'ENDVARS']    
    with open(rofxctlout, 'w') as f:
        for line in lines:
            f.write(line)
            f.write('\n')
    f.close() 
    

    # Capture some variables from this SnowModel simulation
    lines = ['DSET ^etx_'+yrlabel+'.gdat',\
             'UNDEF -9999.0',\
             'XDEF '+nx+' LINEAR '+xll+' '+clsz,\
             'YDEF '+ny+' LINEAR '+yll+' '+clsz,\
             'ZDEF 1 LINEAR 1 1',\
             'TDEF '+str(nt)+' LINEAR '+dt+' 1dy',\
             'VARS 1',\
             'etx 0 0 evapotranspiration (m/time_step)',\
             'ENDVARS']    
    with open(etxctlout, 'w') as f:
        for line in lines:
            f.write(line)
            f.write('\n')
    f.close() 

1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


In [3]:
# load ET data 
# historical ET data
EThist_path = '/nfs/attic/dfh/2020_NPRB/data/climate_na_et/Normal_1991_2020_monthly/'+domain+'/historic_daily_et.nc'
climatena = xr.open_dataset(EThist_path)
#rename coords to match SM
climatena = climatena.rename({'x': 'lon','y': 'lat'})
# reverse y coordinates from .tif file
climatena = climatena.reindex(lat=list(reversed(climatena.lat)))
#convert mm to m 
et_m = climatena.et/1000
# set all pixels to 0 where it is undefined in SM
climatena['et_m'] = et_m

In [4]:
for styr in start_years:
    print(styr)
    #start calibration date    
    st_dt = str(styr)+'-10-01'
    #end calibration date
    ed_dt = str(styr+2)+'-09-30'

    nt = (datetime.strptime(ed_dt,'%Y-%m-%d')-datetime.strptime(st_dt,'%Y-%m-%d')).days+1

    # file naming/organization 
    yrlabel = str(styr+81)+'-'+str(styr+82)
    
    # original runoff ctl 
    rofctl = SMpathOUT+'roff_'+yrlabel+'.ctl'
    # modified runoff ctl
    rofxctlout = SMpathOUT+'rofx_'+yrlabel+'.ctl'
    # modified runoff gdat
    rofxgdatout = SMpathOUT+'rofx_'+yrlabel+'.gdat'
    # modified et ctl
    etxctlout = SMpathOUT+'etx_'+yrlabel+'.ctl'
    # modified et gdat
    etxgdatout = SMpathOUT+'etx_'+yrlabel+'.gdat'
    
    # open runoff
    rf = open_CtlDataset(rofctl)
    # move lat and lon from lower left corner to pixel centers to match ET
    inc = int(clsz)/2
    rf.coords['lat'] = rf['lat']+inc
    rf.coords['lon'] = rf['lon']+inc
    
    # setup steps
    # mask for where sm is undefined
    rfmask=rf.roff.mean(dim='time').values
    # for each date in the model - subtract et
    # model simulation month
    rofm = pd.to_datetime(rf.roff.time.values).month
    # model simulation day
    rofd = pd.to_datetime(rf.roff.time.values).day
    #empty array for new runoff values
    rofxar = np.empty_like(rf.roff)
    #empty array for new et values
    etxar = np.empty_like(rf.roff)
    
    # xr to np arrays 
    R = rf.roff.values
    ET=climatena.et_m.values
    #timestamps
    timecount = rf.time.values
    #day of year for et 
    doyidx = rf.time.dt.dayofyear.values
    for i in range(len(timecount)):
        #print(timecount[i])
        # doy index for et
        idx = doyidx[i]-1
        # select the daily runoff [m]
        rofslice = R[i,:,:]
        # for leap years - repeat the 12/31
        if idx == 365:
            # for leap year - repeat the last day of the year
            etslice = ET[364,:,:]
        else:
            # select et in m for the correct day and month of the year
            etslice = ET[idx,:,:]

        # subtract et from runoff
        dif = rofslice-etslice
        # set negative runoff values to 0 
        difnoneg = np.where(dif>0,dif,0)
        # set undefined values in sm to -9999
        diffinal = np.where(rfmask!=-9999,difnoneg,-9999)
        # add to rofx array
        rofxar[i,:,:] = diffinal
        # et slice where there is not negative runoff
        etadjustslice = np.where(dif>0,etslice,rofslice)
        # add new et values to et array 
        etxar[i,:,:] = etadjustslice
        
    # write rofx data out
    print('writing out rofx data')
    data = rofxar
    # write out new rofx as a grads file
    with open(rofxgdatout, 'wb') as wf:
        float_array = np.float32(data).flatten()
        float_array.tofile(wf)
    wf.close()
    # Function to write out .ctl file for hydroflow steps
    dt = datetime.strptime(st_dt, '%Y-%m-%d').strftime('%M:%SZ%d%b%Y')
    # Capture some variables from this SnowModel simulation
    lines = ['DSET ^../../outputs/wo_assim/rofx_cal.gdat',\
             'UNDEF -9999.0',\
             'XDEF '+nx+' LINEAR '+xll+' '+clsz,\
             'YDEF '+ny+' LINEAR '+yll+' '+clsz,\
             'ZDEF 1 LINEAR 1 1',\
             'TDEF '+str(nt)+' LINEAR '+dt+' 1dy',\
             'VARS 1',\
             'roff 0 0 runoff from snowpack base (m/time_step)',\
             'ENDVARS']    
    with open(rofxctlout, 'w') as f:
        for line in lines:
            f.write(line)
            f.write('\n')
    f.close() 
    
    # write etx data out
    print('writing out etx data')
    data = etxar
    # write out new rofx as a grads file
    with open(etxgdatout, 'wb') as wf:
        float_array = np.float32(data).flatten()
        float_array.tofile(wf)
    wf.close()
    # Function to write out .ctl file for hydroflow steps
    dt = datetime.strptime(st_dt, '%Y-%m-%d').strftime('%M:%SZ%d%b%Y')
    # Capture some variables from this SnowModel simulation
    lines = ['DSET ^../../outputs/wo_assim/etx_cal.gdat',\
             'UNDEF -9999.0',\
             'XDEF '+nx+' LINEAR '+xll+' '+clsz,\
             'YDEF '+ny+' LINEAR '+yll+' '+clsz,\
             'ZDEF 1 LINEAR 1 1',\
             'TDEF '+str(nt)+' LINEAR '+dt+' 1dy',\
             'VARS 1',\
             'etx 0 0 evapotranspiration (m/time_step)',\
             'ENDVARS']    
    with open(etxctlout, 'w') as f:
        for line in lines:
            f.write(line)
            f.write('\n')
    f.close() 

1989
writing out rofx data
writing out etx data
1990
writing out rofx data
writing out etx data
1991
writing out rofx data
writing out etx data
1992
writing out rofx data
writing out etx data
1993
writing out rofx data
writing out etx data
1994
writing out rofx data
writing out etx data
1995
writing out rofx data
writing out etx data
1996
writing out rofx data
writing out etx data
1997
writing out rofx data
writing out etx data
1998
writing out rofx data
writing out etx data
1999
writing out rofx data
writing out etx data
2000
writing out rofx data
writing out etx data
2001
writing out rofx data
writing out etx data
2002
writing out rofx data
writing out etx data
2003
writing out rofx data
writing out etx data
2004
writing out rofx data
writing out etx data
2005
writing out rofx data
writing out etx data
2006
writing out rofx data
writing out etx data
2007
writing out rofx data
writing out etx data
2008
writing out rofx data
writing out etx data
2009
writing out rofx data
writing out e